# Это тестовая задача для Школы менеджеров одной крупной российской IT-компании :).

# Условия задачи

Яндекс.Музыка проводит офлайн-фестиваль самых популярных исполнителей жанра техно. Чтобы привлечь пользователей на мероприятие, маркетолог Иван должен разослать email пользователям Музыки. Отправка одного письма стоит 10 рублей, а цена билета составляет 700 рублей. **Как бы вы посоветовали Ивану сформировать выборку для отправки, учитывая логи прослушиваний пользователей Яндекс.Музыки?**

Обозначения в таблице:

audition_id – ID прослушивания в логе

utc_audition_start_dttm – дата начала прослушивания в формате YYYY-MM-DD hh:mm:ss

utc_audition_end_dttm – дата окончания прослушивания в формате YYYY-MM-DD hh:mm:ss

user_id – ID пользователя Яндекс.Музыки

track_id – ID трека Яндекс.Музыки

track_duration – продолжительность трека в секундах

track_genre – жанр трека

# **Решение задачи:**

# Нужны уточнения
Иван, привет! Для решения задачи не хватает данных. Сколько всего человек нужно привлечь на офлайн-мероприятие? Какой процент от стоимости билета можно потратить на маркетинг? Какова целевая конверсия? Рассылка предполагается только по пользователям из логов? Если конверсия будет маленькой, их может не хватить :).

Я предположил, что на мероприятие нужно привлечь 30-50 человек, на маркетинг можно потратить не больше 20% от стоимости билета, а конверсия писем "любителям техно" в покупку билета на офлайн-мероприятие будет равнятся 10%.

# **Ответ на задачу: Кого приглашаем на офлайн-мероприятие?**

Мы получили 4 группы пользователей. 217 пользователей получили высший балл по "любви к техно". Им нужно отправить приглашение в первую очередь. Во второй группе 267 пользователя, их тоже можно назвать любителями техно. Если ожидаемая конверсия равна 10% и нужно, чтобы на мероприятие пришли 30-50 человек, то рассылки по этим двум группа пользователей хватит - придет, предположительно, около 50 человек. В противном случае можно разослать приглашения группе пользователей с одним баллом по "любви к техно" - этих пользователей 368. Внизу подробно расписал логику, как выделял эти когорты. Если вкратце: нашел тех, кто слушает музыку в жанре техно больше всего (за исключением попа и рока, эти жанры вне конкуренции :)); потом нашел тех, кто входит в топ 35% по общему времени прослушивания музыки в жанре техно - вне зависимости от того, сколько слушали музыку в других жанрах; и третий критерий - вхождение в топ 35% по удельному времени прослушивания техно среди общего времени прослушивания на Яндекс.Музыке (общее время прослушивания музыки в жанре техно разделил на общее время прослушивания в Яндекс.Музыке).

Если согласен с логикой, отпишись, какие пользователи интересны, я вышлю выборку.

# Первый взгляд на данные и предобработка

In [ ]:
import pandas as pd
! gdown --id 1nzztj36UaAEws3ej1yqgoTqOaR8s0Jxj

Downloading...
From: https://drive.google.com/uc?id=1nzztj36UaAEws3ej1yqgoTqOaR8s0Jxj
To: /content/music_data.csv
150MB [00:03, 48.8MB/s]


In [ ]:
df = pd.read_csv('music_data.csv')
df.head()

,Unnamed: 0,audition_id,track_duration,track_genre,track_id,user_id,utc_audition_end_dttm,utc_audition_start_dttm
0,0,3fecd60bf5564de7bb0064335f482b4d,336.629082,RAP,efd614e5-239a-418e-b39c-181b43719b62,a72d73ea124840698aa8,2021-02-28T12:32:10.776000Z,2021-02-28T12:31:01.865000Z
1,1,8ae1703df8354ee6b8d39ce2ceae4508,428.797846,VOCAL,d4697e6e-698a-41e8-8e15-ec890c94751e,066cc9287de340f281ba,2021-01-19T12:13:43.369000Z,2021-01-19T12:12:48.185000Z
2,2,44383125d98a4d939e7f294602674fc6,463.467143,METAL,e006afab-c832-4d07-8cd0-7d4a9c2aabe8,95ed23b52b104f66b4b1,2021-02-27T19:53:57.143000Z,2021-02-27T19:52:42.470000Z
3,3,a90b74992c9f4046a68935cb83ced7ee,323.746259,HARDROCK,c2ea845c-fcba-480a-b41a-d58bf9493375,b0a6dcec6f604b50b37e,2021-02-24T01:51:44.526000Z,2021-02-24T01:46:25.549000Z
4,4,0d98a88fba0a4dc3bb0444089a0ce16b,316.888403,POP,ad68738c-2a38-4e0f-910a-71bb0aa0a0c0,6788b1c78a724de488c9,2020-12-24T05:04:32.855000Z,2020-12-24T05:00:09.645000Z


Лишний столбец 'Unnamed: 0'. Удалим его.

In [ ]:
df = df.drop(columns='Unnamed: 0')
df['track_genre'] = df['track_genre'].str.lower()
df['track_genre'].value_counts(normalize=True)

pop            0.177867
rock           0.116484
rap            0.091641
techno         0.070610
hardrock       0.064504
metal          0.060215
podcasts       0.058958
kpop           0.058664
dubstep        0.056650
vocal          0.050022
electronics    0.048688
rnb            0.024346
disco          0.023179
house          0.021936
industrial     0.018679
dance          0.018319
punk           0.009722
relax          0.009014
jazz           0.007458
blues          0.006709
classical      0.005306
postrock       0.001027
Name: track_genre, dtype: float64

По количеству прослушанных треков жанр техно на 4-м месте по популярности. Не замечено пропущенных или аномальных значений. Значения столбца track_genre привели к строчным буквам.

In [ ]:
df.duplicated().sum()

0

В данных нет дубликатов.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842397 entries, 0 to 842396
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   audition_id              842397 non-null  object 
 1   track_duration           842397 non-null  float64
 2   track_genre              842397 non-null  object 
 3   track_id                 842397 non-null  object 
 4   user_id                  842397 non-null  object 
 5   utc_audition_end_dttm    842397 non-null  object 
 6   utc_audition_start_dttm  842397 non-null  object 
dtypes: float64(1), object(6)
memory usage: 45.0+ MB


Нет пропусков. Данные в столбцах utc_audition_end_dttm и utc_audition_start_dttm типа object. Переведем их в тип datetime, чтобы оценить, как долго пользователи слушают тот или иной трек.

In [ ]:
df['utc_audition_start_dttm'] = pd.to_datetime(df['utc_audition_start_dttm'], 
                                       format='%Y-%m-%dT%H:%M:%S')
df['utc_audition_end_dttm'] = pd.to_datetime(df['utc_audition_end_dttm'], 
                                       format='%Y-%m-%dT%H:%M:%S')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842397 entries, 0 to 842396
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   audition_id              842397 non-null  object             
 1   track_duration           842397 non-null  float64            
 2   track_genre              842397 non-null  object             
 3   track_id                 842397 non-null  object             
 4   user_id                  842397 non-null  object             
 5   utc_audition_end_dttm    842397 non-null  datetime64[ns, UTC]
 6   utc_audition_start_dttm  842397 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(1), object(4)
memory usage: 45.0+ MB


Успешно перевели столбцы utc_audition_end_dttm и utc_audition_start_dttm в формат datetime. Теперь можем узнать, сколько пользователи слушали тот или иной трек. В дальнейшем это понадобится, чтобы считать не число прослушанных треков, а суммарное время, которое пользователи потратили на прослушивание треков того или иного жанра. Кажется, это так надежнее определять, насколько людям нравится тот или иной жанр музыки.

In [ ]:
df['audition_duration'] = df['utc_audition_end_dttm'] - df['utc_audition_start_dttm']
df['audition_duration'] = df['audition_duration'].dt.total_seconds()
df.head()

,audition_id,track_duration,track_genre,track_id,user_id,utc_audition_end_dttm,utc_audition_start_dttm,audition_duration
0,3fecd60bf5564de7bb0064335f482b4d,336.629082,rap,efd614e5-239a-418e-b39c-181b43719b62,a72d73ea124840698aa8,2021-02-28 12:32:10.776000+00:00,2021-02-28 12:31:01.865000+00:00,68.911
1,8ae1703df8354ee6b8d39ce2ceae4508,428.797846,vocal,d4697e6e-698a-41e8-8e15-ec890c94751e,066cc9287de340f281ba,2021-01-19 12:13:43.369000+00:00,2021-01-19 12:12:48.185000+00:00,55.184
2,44383125d98a4d939e7f294602674fc6,463.467143,metal,e006afab-c832-4d07-8cd0-7d4a9c2aabe8,95ed23b52b104f66b4b1,2021-02-27 19:53:57.143000+00:00,2021-02-27 19:52:42.470000+00:00,74.673
3,a90b74992c9f4046a68935cb83ced7ee,323.746259,hardrock,c2ea845c-fcba-480a-b41a-d58bf9493375,b0a6dcec6f604b50b37e,2021-02-24 01:51:44.526000+00:00,2021-02-24 01:46:25.549000+00:00,318.977
4,0d98a88fba0a4dc3bb0444089a0ce16b,316.888403,pop,ad68738c-2a38-4e0f-910a-71bb0aa0a0c0,6788b1c78a724de488c9,2020-12-24 05:04:32.855000+00:00,2020-12-24 05:00:09.645000+00:00,263.210


Добавили столбец audition_duration, который показывает, сколько времени в секундах тот или иной пользователь прослушивал трек. 

In [ ]:
df['audition_duration'].describe()

count    842397.000000
mean        425.734771
std         696.770853
min           0.005000
25%         128.322000
50%         316.265000
75%         397.080000
max        5854.082000
Name: audition_duration, dtype: float64

Средняя длина трека - 425 секунд. Медиана меньше на 100 секунд. Видимо, есть много треков с аномальной длительностью.

In [ ]:
df.groupby('track_genre')['audition_duration'].mean()

track_genre
blues           378.646864
classical       602.531504
dance           245.626371
disco           259.854857
dubstep         327.786576
electronics     482.112547
hardrock        213.674936
house           195.281813
industrial      183.320284
jazz            375.965649
kpop            273.505743
metal           294.681312
podcasts       2750.761483
pop             245.381340
postrock        848.230401
punk            201.245290
rap             234.238335
relax           656.832323
rnb             248.735326
rock            271.609139
techno          307.520752
vocal           277.904318
Name: audition_duration, dtype: float64

Видим, что подкасты слушают очень долго. Это не удивительно. Удалим подкасты из датасета, чтобы они не искажали данные по средним прослушиваниям. Да и подкасты - не жанр музыки :).

In [ ]:
df = df[df['track_genre'] != 'podcasts']
df['audition_duration'].describe()

count    792731.000000
mean        280.067733
std         168.973679
min           0.005000
25%         116.790500
50%         306.101000
75%         381.129000
max        1624.539000
Name: audition_duration, dtype: float64

Теперь среднее незначительно отличается от к медианы.

In [ ]:
df['utc_audition_start_dttm'].describe(datetime_is_numeric=True)

count                                 792731
mean     2021-02-09 16:01:45.403722752+00:00
min         2020-06-08 13:47:24.028000+00:00
25%      2021-02-01 22:26:50.641500160+00:00
50%      2021-02-15 19:22:20.849999872+00:00
75%      2021-02-23 21:43:30.481999872+00:00
max         2021-03-01 15:59:56.310000+00:00
Name: utc_audition_start_dttm, dtype: object

Данные в датасете за последние 9 месяцев. Вряд ли вкусы пользователей за это время изменились. Данные по дате прослушивания фильтровать не будем.

# Анализируем жанровые предпочтения пользователей Яндекс.Музыки

In [ ]:
df2 = df.pivot_table(index='user_id', columns='track_genre', values='audition_duration', aggfunc='sum')
df2 = df2.fillna(0)
df2

track_genre,blues,classical,dance,disco,dubstep,electronics,hardrock,house,industrial,jazz,kpop,metal,pop,postrock,punk,rap,relax,rnb,rock,techno,vocal
user_id,,,,,,,,,,,,,,,,,,,,,
002123faf66b47b5aafa,585.254,1771.157,2912.786,2074.449,4835.516,6212.613,3641.298,1394.675,1613.431,1600.949,4963.418,7947.370,13877.272,0.000,1432.666,10199.228,1518.297,3343.901,8852.924,6398.402,6173.461
0041383820e248e487a6,3193.866,4755.536,4771.090,4058.303,24182.167,16742.056,14259.960,3710.053,3517.560,1706.269,16123.939,18657.958,47693.333,0.000,2411.794,24436.918,7411.519,7738.389,31331.417,22007.036,15389.733
008005a451764a63aa3f,2730.923,777.504,3283.055,4270.523,14660.465,12287.735,7459.531,1109.824,1926.677,616.239,7842.902,8282.051,31897.340,1981.802,1407.738,16279.067,4627.707,2819.856,18642.996,12601.688,9419.729
00c017dc34254d39a390,573.271,2088.069,2963.028,3789.324,11690.230,20521.883,7896.802,2911.561,3477.967,1056.138,10382.012,10248.488,27292.729,1362.029,1235.734,12086.540,3022.802,3453.940,22045.327,17279.222,9052.995
00f628a2f8044ae18d88,757.769,862.389,2525.456,2006.748,5912.211,6963.513,5460.906,1209.009,1298.013,1125.515,6758.741,7798.531,18720.816,1136.704,1208.157,7781.424,2076.856,3352.064,13620.023,11253.504,3514.723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff77d34a23c04d36a421,527.694,0.000,2680.352,1462.920,9881.080,11243.106,4810.117,390.542,1791.716,1923.160,7838.735,6381.228,18517.957,0.000,2992.678,7884.278,2573.665,2818.227,9785.139,9823.367,5919.949
ff9b606118d449f1ab15,288.218,983.992,1827.100,3084.156,7638.306,5457.963,4591.672,1835.326,1183.003,920.518,6430.113,9423.335,17171.940,0.000,263.576,9213.471,3097.763,2812.541,15253.111,8166.387,6754.817
ffa2421c514844c087f3,1261.529,936.639,2598.192,4255.150,6408.833,16967.290,10354.062,2377.614,2683.381,2349.352,10862.341,9388.858,24423.004,0.000,1226.254,17653.320,6317.256,5756.655,18526.114,8799.071,9393.394


Получили таблицу, где для каждого пользователя показано, сколько всего он слушал композиции того или иного жанра в секундах. Найдем для каждого любимый жанр, композиции которого пользователь слушал дольше всего. Заметим, что в логе у нас 1727 пользователей.

In [ ]:
df2['favorite_genre'] = df2.idxmax(axis=1)

In [ ]:
df2['favorite_genre'].value_counts()

pop            1624
rock             91
electronics      11
techno            1
Name: favorite_genre, dtype: int64

Видим, что только один пользователь дольше всего слушал композиции жанра техно. Уберем из таблицы столбец pop - видимо, с ним по общей длительности прослушивания техно тягаться не может :).

In [ ]:
df2 = df2.drop(columns=['pop', 'favorite_genre'])
df2['favorite_genre'] = df2.idxmax(axis=1)

In [ ]:
df2['favorite_genre'].value_counts()

rock           1384
electronics     198
techno           69
rap              39
dubstep          19
metal            15
kpop              3
Name: favorite_genre, dtype: int64

Получается, что 69 пользователей из 1727 любят техно так, что слушают его остальных жанров за вычетом попа и подкастов. Это около 4% пользователей из выборки. Попробуем убрать рок.

In [ ]:
df2 = df2.drop(columns=['rock', 'favorite_genre'])
df2['favorite_genre'] = df2.idxmax(axis=1)
df2['favorite_genre'].value_counts()

electronics    755
techno         398
rap            335
dubstep        114
metal           94
kpop            25
vocal            3
hardrock         3
Name: favorite_genre, dtype: int64

In [ ]:
techno_favorite_genre_users = df2[df2['favorite_genre'] == 'techno'].reset_index()['user_id']
techno_favorite_genre_users

0      00f628a2f8044ae18d88
1      016ab75e22374843b684
2      030cb42f156b4bac9607
3      0311bed05d2b4c7a8ab9
4      03f59562ca5c428a9425
               ...         
393    fa06379b3b3a4144afca
394    fbc74bb744cc46b3a401
395    fcc5a11f401b4fb8a93f
396    fd4974a26a7344cf9655
397    ff122fc8c3354f059efa
Name: user_id, Length: 398, dtype: object

398 пользователя предпочитают техно всем остальным жанрам кроме поп-музыки и рока. Это 23% от выборки, сохранили идентификаторы этих пользователей в techno_favorite_genre_users. Если конверсия будет равнятся 10%, то можем ожидать, что на мероприятие придет около 40 человек. Что если конверсия будет меньше? Тогда нам нужно понять, кому слать приглашения. Отранжируем всех пользователей по их любви к техно.

In [ ]:
df_users = df.groupby('user_id')['audition_duration'].sum().fillna(0).reset_index()
techno_duration_total = df[df['track_genre'] == 'techno'].groupby('user_id')['audition_duration'].sum().fillna(0).reset_index()
df_users = pd.merge(left=df_users, right=techno_duration_total, how='left', on='user_id')
df_users.columns = ['user_id', 'audition_duration_total', 'audition_duration_techno']
df_users.head()

,user_id,audition_duration_total,audition_duration_techno
0,002123faf66b47b5aafa,91349.067,6398.402
1,0041383820e248e487a6,274098.896,22007.036
2,008005a451764a63aa3f,164925.352,12601.688
3,00c017dc34254d39a390,174430.091,17279.222
4,00f628a2f8044ae18d88,105343.072,11253.504


Получили таблицу с тремя колонками: идентификатор пользователя, общее время прослушивания музыки в сервисе, общее время прослушивания музыки в жанре техно. Добавим колонку, которая будет показывать долю техно в общем времени прослушивания. Назовем эту метрику techno_love_score.

In [ ]:
df_users['techno_love_score'] = df_users['audition_duration_techno'] / df_users['audition_duration_total']
df_users['techno_love_score'].describe()

count    1727.000000
mean        0.082359
std         0.016351
min         0.013144
25%         0.071485
50%         0.081886
75%         0.091641
max         0.152903
Name: techno_love_score, dtype: float64

В среднем около 8% всего времени прослушивания музыки занимает техно. Мы можем взять около 35% пользователей с самым высоким techno_love_score.

In [ ]:
df_users['top_35_percent_techno_score'] = (df_users['techno_love_score'] > df_users['techno_love_score'].quantile(q=0.65))*1
df_users['top_35_percent_techno_score'].sum()

605

Здесь получили 605 пользователей. Даже если конверсия будет 7%, то все равно получим 40 человек на мероприятии. Попробуем еще проранжировать пользователей по общему времени прослушивания музыки в жанре техно.

In [ ]:
df_users['top_35_percent_techno_total_duration'] = (df_users['audition_duration_techno'] > df_users['audition_duration_techno'].quantile(q=0.65))*1
df_users.head()

,user_id,audition_duration_total,audition_duration_techno,techno_love_score,top_35_percent_techno_score,top_35_percent_techno_total_duration
0,002123faf66b47b5aafa,91349.067,6398.402,0.070043,0,0
1,0041383820e248e487a6,274098.896,22007.036,0.080289,0,1
2,008005a451764a63aa3f,164925.352,12601.688,0.076408,0,1
3,00c017dc34254d39a390,174430.091,17279.222,0.099061,1,1
4,00f628a2f8044ae18d88,105343.072,11253.504,0.106827,1,0


В столбце top_35_percent_techno_total_duration стоит 1, если пользователь входит в топ 35% по общему прослушивания музыки жанра техно и 0 в противном случае.

In [ ]:
df_users['techno_favorite_genre'] = df_users['user_id'].isin(techno_favorite_genre_users.unique())*1
df_users.head()

,user_id,audition_duration_total,audition_duration_techno,techno_love_score,top_35_percent_techno_score,top_35_percent_techno_total_duration,techno_favorite_genre
0,002123faf66b47b5aafa,91349.067,6398.402,0.070043,0,0,0
1,0041383820e248e487a6,274098.896,22007.036,0.080289,0,1,0
2,008005a451764a63aa3f,164925.352,12601.688,0.076408,0,1,0
3,00c017dc34254d39a390,174430.091,17279.222,0.099061,1,1,0
4,00f628a2f8044ae18d88,105343.072,11253.504,0.106827,1,0,1


Добавили в таблицу еще столбец techno_favorite_genre, который равен 1, если техно - любимый жанр пользователя (не считая поп- и рок-музыки), и 0 в противном случае. У нас получилось 3 метрики. Создадим столбец, который будет давать итоговую оценку того, насколько пользователен заинтересован музыкой жанра техно. 3 - высшая оценка, 0 - низшая.

In [ ]:
df_users['final_score'] = df_users['top_35_percent_techno_score'] + df_users['top_35_percent_techno_total_duration'] + df_users['techno_favorite_genre']
df_users['final_score'].value_counts()

0    835
1    393
2    282
3    217
Name: final_score, dtype: int64

In [ ]:
df_users['audition_duration_total'].describe()

count      1727.000000
mean     128557.252053
std       37805.925794
min       19397.650000
25%      104063.383000
50%      127700.133000
75%      153557.380000
max      274098.896000
Name: audition_duration_total, dtype: float64

В выборке только постоянные пользователи Яндекс.Музыки, так как минимальное значение общего времени прослушивания - около 5 часов. Однако это в 5 раз меньше медианного значения. Скорее всего, стоит удалить 5% пользователей с минимальным общим временем прослушивания.

In [ ]:
quant = df_users['audition_duration_total'].quantile(q=0.05)
quant

66152.34429999997

5-й перцентиль равен 66152 секундам или около 18 часам и находится всего в одном стандартном отклонении от медианы. Оставим всех пользователей, кто пользовался сервисом дольше.

In [ ]:
df_users = df_users[df_users['audition_duration_total'] > quant]
df_users['final_score'].value_counts()

0    788
1    368
2    267
3    217
Name: final_score, dtype: int64

# Рекомендации:
Мы получили 4 группы пользователей. 217 пользователей получили высший балл по "любви к техно". Им нужно отправить приглашение в первую очередь. Во второй группе 267 пользователя, их тоже можно назвать любителями техно. Если ожидаемая конверсия равна 10% и нужно, чтобы на мероприятие пришли 30-50 человек, то рассылки по этим двум группа пользователей хватит. В противном случае можно разослать приглашения группе пользователей с одним баллом по "любви к техно" - их 368.

# Общие выводы по работе с данными:

- Данные хорошие: нет дубликатов, пропущенных или аномальных значений
- Данные свежие: за последние 9 месяцев
- Минимальное значение общего времени прослушивания пользователя равно 5 часов, медианное - около 25 часов. Из выборки удалил пользователей, общее время прослушивания музыки на сервисе которых меньше 18 часов - это примерно 5% пользователей в логах.
- Удалил из выборки треки с жанром podcasts, потому что они не относятся к музыкальному жанру и у них аномально долгое время прослушивания.
- Каждому пользователю присвоил 3 оценки: входит в топ 35% по времени прослушивания музыки жанра техно (1 балл), входит в топ 35% по удельному времени прослушивания музыки жанра техно относительно общего времени прослушивания (1 балл), жанр техно - самый популярный жанр (исключая поп- и рок-музыку) по общему времени прослушивания.
- 3 балла набрали 217 пользователей, 2 - 267, 1 - 368. Без баллов остались 788 пользователей. Рассылки, кажется, лучше делать сначала по тем, у кого больше баллов.